In [2]:
import pandas as pd
import numpy as np
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import datetime

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

from statsmodels.formula.api  import ols
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras import layers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline



In [5]:

topicdata = pd.read_csv(r'/Users/cairo/Google Drive/wechat data/TopicOutcomeAll20Topic.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
#topicdata = pd.read_csv(r'C:/Users/Junhao/Google Drive/wechat data/TopicOutcomeAll20Topic.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
 

In [6]:
topicdata = topicdata.head(1000) #use only 1000 data here

In [20]:

#Q1: Which topics can predict the best influencer performance (indicated by likes and clicks of posts) in different lifespan of the account; what is the optimal level of topic diversity?

#Q4: To what extent should an account follow the hot topics in news? What is the optimal balance(i.e., topic proportion)  of following trending topics vs. sticking to account expertise? For instance, should an account on fitness share articles about the US election? (This might need extra data about hot news topic in Chinese media)


##################################
#linear regression and XGboost
##################################


###############################################################################################
#prepare data
topicdata.head()


y = topicdata.likeCount

#X = topicdata[topicdata.columns[-30:]]
X0 = topicdata.iloc[:,-20:]
X1 = topicdata[["clicksCount", "orderNum", "originalFlag"]]

X = pd.concat([X0, X1], axis=1)
X = sm.add_constant(X) #add a intercept term to the regression
#X.reset_index(drop=True)

#X.index = pd.RangeIndex(len(X.index))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
X_test = X_test.astype(float)
X_train = X_train.astype(float)
y_train = y_train.astype(float)
y_test = y_test.astype(float)


In [8]:

###############################################################################################
#linear regression with all variables
results = sm.OLS(y.astype(float), X.astype(float)).fit()
results.summary().tables[0]


/Users/cairo/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1830: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
/Users/cairo/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse


Dep. Variable:,likeCount,R-squared:,0.393
Model:,OLS,Adj. R-squared:,0.380
Method:,Least Squares,F-statistic:,28.81
Date:,"Sun, 24 Jan 2021",Prob (F-statistic):,1.85e-90
Time:,19:39:31,Log-Likelihood:,-7154.8
No. Observations:,1000,AIC:,1.436e+04
Df Residuals:,977,BIC:,1.447e+04
Df Model:,22,,
Covariance Type:,nonrobust,,


In [9]:
#linear regression with only the topic variables
results2 = sm.OLS(y.astype(float), X0.astype(float)).fit()
results2.summary().tables[0]

Dep. Variable:,likeCount,R-squared (uncentered):,0.178
Model:,OLS,Adj. R-squared (uncentered):,0.162
Method:,Least Squares,F-statistic:,10.64
Date:,"Sun, 24 Jan 2021",Prob (F-statistic):,1.53e-30
Time:,19:40:28,Log-Likelihood:,-7481.1
No. Observations:,1000,AIC:,1.500e+04
Df Residuals:,980,BIC:,1.510e+04
Df Model:,20,,
Covariance Type:,nonrobust,,


In [10]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)
reg.score(X_train, y_train)

0.37008656615804936

In [14]:
from sklearn.metrics import mean_squared_error, r2_score
preds = reg.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test.astype(float), preds))
print("RMSE: %f" % (rmse))

RMSE: 173.084546


In [13]:
###############################################################################################
#xgboost regression
from xgboost.sklearn import XGBRegressor
import xgboost as xgb

xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

xg_reg.fit(X_train.astype(float),y_train.astype(float))

preds = xg_reg.predict(X_test.astype(float))

rmse = np.sqrt(mean_squared_error(y_test.astype(float), preds))
print("RMSE: %f" % (rmse))




[19:42:37] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
RMSE: 219.777391


In [17]:
###############################################################################################
#######################
#regression in neural networks with keras
######################
input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(64, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='relu'))

model.compile(loss='mean_squared_error', 
              optimizer='adam',
              metrics=['mean_absolute_percentage_error']
              
              )
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 10)                250       
_________________________________________________________________
dense_7 (Dense)              (None, 64)                704       
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 65        
Total params: 1,019
Trainable params: 1,019
Non-trainable params: 0
_________________________________________________________________


In [21]:

history = model.fit(X_train , y_train,
                    epochs=5,
                    verbose=True,
                    #steps_per_epoch=5,            
                    validation_data=(X_test, y_test),
                    batch_size=100
                    )

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

Epoch 1/5
8/8 [==============================] - 5s 546ms/step - loss: 718752.9931 - mean_absolute_percentage_error: 265017824.0000 - val_loss: 123557.4375 - val_mean_absolute_percentage_error: 99.9243
Epoch 2/5
8/8 [==============================] - 0s 9ms/step - loss: 225782.8776 - mean_absolute_percentage_error: 41982745.2222 - val_loss: 123557.4609 - val_mean_absolute_percentage_error: 100.0000
Epoch 3/5
8/8 [==============================] - 0s 8ms/step - loss: 263834.0260 - mean_absolute_percentage_error: 29543157.3333 - val_loss: 123557.4609 - val_mean_absolute_percentage_error: 100.0000
Epoch 4/5
8/8 [==============================] - 0s 9ms/step - loss: 231932.8160 - mean_absolute_percentage_error: 15541388.0451 - val_loss: 123557.4609 - val_mean_absolute_percentage_error: 100.0000
Epoch 5/5
8/8 [==============================] - 0s 7ms/step - loss: 264879.8681 - mean_absolute_percentage_error: 30440528.5556 - val_loss: 123557.4609 - val_mean_absolute_percentage_error: 100.000

,loss,mean_absolute_percentage_error,val_loss,val_mean_absolute_percentage_error,epoch
0,427565.187500,148670080.0,123557.437500,99.924339,0
1,249481.703125,53744992.0,123557.460938,100.000000,1
2,249481.484375,37065776.0,123557.460938,100.000000,2
3,249481.359375,26863070.0,123557.460938,100.000000,3
4,249481.312500,22129162.0,123557.460938,100.000000,4
